In [22]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("./Companies_Analyzed.csv")

# Extract the CIK column
cik_column = df['CIK']

# Initialize an empty set to store unique CIKs
unique_ciks = set()

# Iterate through the CIK column
for entry in cik_column:
    # Split the entry by semicolon and add each CIK to the set
    if pd.notna(entry):  # Check if the entry is not NaN
        ciks = entry.split(';')
        unique_ciks.update(ciks)

# Convert the set to a list (optional)
unique_ciks_list = list(unique_ciks)
#unique_ciks_list = [ x for x in unique_ciks_list if x.isdigit() ]
#unique_ciks_list = [ int(x) for x in unique_ciks_list ]
fixed_ciks = [num.strip().zfill(10) for num in unique_ciks_list]
print(fixed_ciks)

with open("./CoAn.txt", 'w') as f:
    for i in fixed_ciks:
        f.write(f"{i}\n")

['0000015615', '0001316835', '0001001902', '0000884624', '0001897982', '0000946563', '0001371782', '0000879635', '0000064472', '0001059993', '0001434754', '0000810383', '0001053706', '0001036262', '0000894158', '0001334325', '0001690666', '0000708955', '0001429937', '0001088856', '0000726601', '0000002178', '0000890066', '0000813672', '0001296445', '0000872589', '0001588823', '0000896156', '0001591188', '0001405196', '0001369241', '0000016058', '0001082038', '0001138639', '0000049754', '0001377630', '0000350868', '0000029002', '0001368265', '0001291000', '0001362004', '0001084765', '0001362468', '0001363829', '0000886136', '0000007623', '0000016859', '0001392972', '0001029199', '0000085535', '0001039065', '0001964333', '0000920427', '0000890541', '0001422427', '0000933738', '0000874663', '0001111928', '0000793628', '0001305597', '0000082811', '0000019745', '0000832428', '0000876427', '0000923796', '0000025475', '0000012239', '0001000045', '0000937556', '0001071255', '0000351789', '0000

In [9]:
import pandas as pd

# Load the lookup table
lookup_df = pd.read_csv("C:/Users/feldberg.dartmouth/Downloads/rawdata/cik_name_lookup.csv", sep='\t')
lookup_df = lookup_df.drop_duplicates(subset=['cw_id'])
lookup_df.drop(columns=['row_id', 'edgar_name', "match_name"], inplace = True)
lookup_df.dropna(subset=['cik', 'cw_id'], inplace=True)
lookup_df = lookup_df.astype('int32')

# Load the company relations data
relations_df = pd.read_csv('C:/Users/feldberg.dartmouth/Downloads/rawdata/company_relations.csv', sep='\t')
relations_df.drop(columns=['relation_id', 'relation_type', 'relation_origin', 'origin_id'], inplace = True)
relations_df.dropna(subset=['source_cw_id', 'target_cw_id'], inplace=True)
relations_df = relations_df.astype('int32')

# Create a dictionary for cw_id to CIK mapping
cw_to_cik = dict(zip(lookup_df['cw_id'], lookup_df['cik']))

# Replace source_cw_id and target_cw_id with corresponding CIK values
relations_df['source_cik'] = relations_df['source_cw_id'].map(cw_to_cik)
relations_df['target_cik'] = relations_df['target_cw_id'].map(cw_to_cik)

# Drop the original cw_id columns if no longer needed
relations_df.drop(columns=['source_cw_id', 'target_cw_id'], inplace=True)

filtered_relations_df = relations_df[
    relations_df['source_cik'].isin(unique_ciks_list) | 
    relations_df['target_cik'].isin(unique_ciks_list)
]

# Save the updated dataframe to a new CSV file
filtered_relations_df.to_csv('C:/Users/feldberg.dartmouth/Downloads/rawdata/updated_company_relations_bw.csv', index=False)

In [10]:
import pandas as pd

# Load the lookup table
lookup_df = pd.read_csv("C:/Users/feldberg.dartmouth/Downloads/rawdata/CIK_CompanyID.csv")
lookup_df = lookup_df.drop_duplicates(subset=['BoardID', 'CIKCode'])
lookup_df.drop(columns=['BoardName'], inplace = True)
lookup_df.dropna(subset=['CIKCode', 'BoardID'], inplace=True)
lookup_df = lookup_df.astype('int32')


# Load the company relations data
relations_df = pd.read_csv('C:/Users/feldberg.dartmouth/Downloads/rawdata/Board_Listed_Associations.csv')
relations_df = relations_df[relations_df.columns.intersection(['OverlapYearStart', 'OverlapYearEnd', 'BoardID', 'CompanyID'])]
relations_df.dropna(subset=['OverlapYearStart', 'OverlapYearEnd', 'BoardID', 'CompanyID'], inplace=True)
relations_df.replace('Curr', 2024, inplace=True)
relations_df = relations_df.astype('int32')

# Create a dictionary for cw_id to CIK mapping
BoardID_to_cik = dict(zip(lookup_df['BoardID'], lookup_df['CIKCode']))

# Replace source_cw_id and target_cw_id with corresponding CIK values
relations_df['source_cik'] = relations_df['BoardID'].map(BoardID_to_cik)
relations_df['target_cik'] = relations_df['CompanyID'].map(BoardID_to_cik)

# Drop the original cw_id columns if no longer needed
relations_df.drop(columns=['BoardID', 'CompanyID'], inplace=True)

# Filter to only data of interest
filtered_relations_df = relations_df[
    relations_df['source_cik'].isin(unique_ciks_list) | 
    relations_df['target_cik'].isin(unique_ciks_list)
]

# Save the updated dataframe to a new CSV file
filtered_relations_df.to_csv('C:/Users/feldberg.dartmouth/Downloads/rawdata/updated_company_relations_bx.csv', index=False)



In [11]:
df_bw = pd.read_csv('C:/Users/feldberg.dartmouth/Downloads/rawdata/updated_company_relations_bw.csv')
df_bx = pd.read_csv('C:/Users/feldberg.dartmouth/Downloads/rawdata/updated_company_relations_bx.csv')

def expand_years(row):
    years = range(row['OverlapYearStart'], row['OverlapYearEnd'] + 1)
    return pd.DataFrame({
        'year': years,
        'source_cik': row['source_cik'],
        'target_cik': row['target_cik']
    })

# Drop rows with missing values in the relevant columns
df_bx_cleaned = df_bx.dropna(subset=['source_cik', 'target_cik'])
df_bw_cleaned = df_bw.dropna(subset=['source_cik', 'target_cik'])

# Ensure all relevant columns are integers
df_bx_cleaned['source_cik'] = df_bx_cleaned['source_cik'].astype(int)
df_bx_cleaned['target_cik'] = df_bx_cleaned['target_cik'].astype(int)

df_bw_cleaned['source_cik'] = df_bw_cleaned['source_cik'].astype(int)
df_bw_cleaned['target_cik'] = df_bw_cleaned['target_cik'].astype(int)

# Expand df_bx_cleaned
expanded_bx_cleaned = pd.concat([expand_years(row) for _, row in df_bx_cleaned.iterrows()], ignore_index=True)
expanded_bx_cleaned['relationship'] = 'subsidiary'

# Add relationship column to df_bw_cleaned
df_bw_cleaned['relationship'] = 'interlock'

# Combine the dataframes
combined_df_cleaned = pd.concat([df_bw_cleaned[['year', 'source_cik', 'target_cik', 'relationship']], expanded_bx_cleaned], ignore_index=True)
combined_df_cleaned = combined_df_cleaned[(combined_df_cleaned['year'] >= 2010) & (combined_df_cleaned['year'] <= 2023)]
combined_df_cleaned = combined_df_cleaned.drop_duplicates()



C:\Users\feldberg.dartmouth\AppData\Local\Temp\ipykernel_9156\1439550904.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bx_cleaned['source_cik'] = df_bx_cleaned['source_cik'].astype(int)
C:\Users\feldberg.dartmouth\AppData\Local\Temp\ipykernel_9156\1439550904.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bx_cleaned['target_cik'] = df_bx_cleaned['target_cik'].astype(int)
C:\Users\feldberg.dartmouth\AppData\Local\Temp\ipykernel_9156\1439550904.py:20: SettingWithCopyWarning: 
A value is tryi

In [12]:
import networkx as nx
import pickle

# Create a dictionary to store the graphs for each year
graphs = {}

# Create graphs for each year
for year in combined_df_cleaned['year'].unique():
    G = nx.MultiDiGraph()
    year_data = combined_df_cleaned[combined_df_cleaned['year'] == year]
    
    for _, row in year_data.iterrows():
        G.add_edge(row['source_cik'], row['target_cik'], relationship=row['relationship'])
    
    graphs[year] = G

# Define the base path for saving the pickle files
base_path = './Pickle Graphs3/'  # Change this to your desired path

# Iterate through each year and graph in the dictionary
for year, graph in graphs.items():
    # Define the filename for each graph
    filename = f'{base_path}/graph_{year}.pkl'
    
    # Save the graph to a pickle file
    with open(filename, 'wb') as f:
        pickle.dump(graph, f)

    print(f'Graph for year {year} saved to {filename}')


Graph for year 2010 saved to ./Pickle Graphs3//graph_2010.pkl
Graph for year 2011 saved to ./Pickle Graphs3//graph_2011.pkl
Graph for year 2012 saved to ./Pickle Graphs3//graph_2012.pkl
Graph for year 2013 saved to ./Pickle Graphs3//graph_2013.pkl
Graph for year 2014 saved to ./Pickle Graphs3//graph_2014.pkl
Graph for year 2015 saved to ./Pickle Graphs3//graph_2015.pkl
Graph for year 2016 saved to ./Pickle Graphs3//graph_2016.pkl
Graph for year 2017 saved to ./Pickle Graphs3//graph_2017.pkl
Graph for year 2018 saved to ./Pickle Graphs3//graph_2018.pkl
Graph for year 2019 saved to ./Pickle Graphs3//graph_2019.pkl
Graph for year 2020 saved to ./Pickle Graphs3//graph_2020.pkl
Graph for year 2022 saved to ./Pickle Graphs3//graph_2022.pkl
Graph for year 2023 saved to ./Pickle Graphs3//graph_2023.pkl
Graph for year 2021 saved to ./Pickle Graphs3//graph_2021.pkl
